In [ ]:
import pandas as pd
from tbdynamics.plotting import plot_model_vs_actual
import nevergrad as ng

# Import our convenience wrapper
from estival.wrappers.nevergrad import optimize_model
from tbdynamics.calib_utils import get_bcm, load_targets
from multiprocessing import cpu_count
import pymc as pm
import numpy as np

In [ ]:
pd.options.plotting.backend = "plotly"

In [ ]:
params = {
    "start_population_size": 2300000.,
    "seed_time": 1810.,
    "seed_num": 10.,
    "seed_duration": 3.,
    "screening_scaleup_shape": 0.1,
    "screening_inflection_time": 1993.,
    "screening_end_asymp": 0.65,
}

bcm = get_bcm(params)

In [ ]:
num_cores = cpu_count()
# Set the number of workers for parallel optimization
opt_class = ng.optimizers.TwoPointsDE
orunner = optimize_model(bcm, opt_class=opt_class, num_workers=num_cores, ci=0.67)
for i in range(8):
    rec = orunner.minimize(1000)
mle_params = rec.value[1]
mle_params
   

In [ ]:
res = bcm.run(mle_params)
derived_df_0 = res.derived_outputs
targets = load_targets()

In [ ]:
plot_model_vs_actual(
    derived_df_0, targets['total_population'], "total_population", "Population", "Modelled vs Data"
)

In [ ]:
plot_model_vs_actual(
    derived_df_0, targets['percentage_latent'], "percentage_latent", "Percentage latent", "Modelled vs Data"
)

In [ ]:
plot_model_vs_actual(derived_df_0, targets['prevalence_pulmonary'], 'prevalence_pulmonary', 'Infectious prevalence', 'Modelled vs Estimation from 2017 prevalence survey')

In [ ]:
plot_model_vs_actual(
    derived_df_0, targets['incidence'], "incidence", "Incidence", "Modelled vs Data"
)

In [ ]:
plot_model_vs_actual(
    derived_df_0, targets['notification'], "notification", "Notifications", "Modelled vs Data"
)

In [ ]:
derived_df_0['cdr'].plot()

In [ ]:
from autumn.infrastructure.remote import springboard
rts = springboard.task.RemoteTaskStore()
rts.cd('projects/tbdynamics/s10041_calibration')
rts.ls()
mt = rts.get_managed_task('2024-04-10T1335-d50k-t20k-b20k')
mt.download_all()